In [1]:
import intersect
import pandas as pd
import geopandas as gpd
import fiona.crs
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

In [2]:
EPSG = 2263

CRS = {
    'proj': 'latlong',
    'init': 'epsg:{:d}'.format(EPSG)
}

In [ ]:
raw_people = pd.read_csv("/Users/asiega/Desktop/CVHpeople.csv")

In [ ]:
raw_people.head(2)

In [3]:
people = intersect.people_df("/Users/asiega/Desktop/CVHpeople.csv", crs=CRS)

In [4]:
people.head(2)

,Internal Contact ID,Latitude,Longitude,geometry
0,5,40.687482,-73.963384,POINT (-73.96338399999999 40.687482)
1,226,40.769909,-73.992111,POINT (-73.99211099999999 40.76990900000001)


In [5]:
def explode(indata):
    """Break down multipolygons in geojson to single polygons per row"""
    indf = gpd.GeoDataFrame.from_file(indata)
    outdf = gpd.GeoDataFrame(columns=indf.columns)
    for idx, row in indf.iterrows():
        if type(row.geometry) == Polygon:
            outdf = outdf.append(row,ignore_index=True)
        if type(row.geometry) == MultiPolygon:
            multdf = gpd.GeoDataFrame(columns=indf.columns)
            recs = len(row.geometry)
            multdf = multdf.append([row]*recs,ignore_index=True)
            for geom in range(recs):
                multdf.loc[geom,'geometry'] = row.geometry[geom]
            outdf = outdf.append(multdf,ignore_index=True)
    return outdf

# raw_shapes = explode("/Users/asiega/Desktop/NYCHA.geojson")

In [12]:
def shapes_df(path):
#     raw_shapes = gpd.read_file(path)
    raw_shapes = explode(path)
    raw_shapes.crs = fiona.crs.from_epsg(EPSG)
    zones = raw_shapes.to_crs(fiona.crs.from_epsg(EPSG))
    zones_crs = zones.to_crs(CRS)
    return zones_crs

In [13]:
shapes = shapes_df("/Users/asiega/Desktop/NYCHA.geojson")

In [18]:
people["geometry"].sort_values().head()

1        POINT (-73.99211099999999 40.76990900000001)
11421            POINT (-73.84563799999999 40.819266)
11420            POINT (-73.846222 40.81932800000001)
11419            POINT (-73.98685500000001 40.574812)
0                POINT (-73.96338399999999 40.687482)
Name: geometry, dtype: object

In [17]:
shapes["geometry"].sort_values().head()

851    POLYGON ((-73.79061904700808 40.66815582089683...
854    POLYGON ((-73.79838341966898 40.67181201846752...
852    POLYGON ((-73.80503351710118 40.66845837598144...
853    POLYGON ((-73.80219003598181 40.67066307272878...
850    POLYGON ((-73.80003796052573 40.59850693799366...
Name: geometry, dtype: object

In [16]:
intersect.merge(shapes.head(50), people)

ValueError: need at least one array to concatenate

In [ ]:
raw_shapes.crs = CRS
zones = raw_shapes.to_crs(people.crs)

In [ ]:
zones.head()

In [ ]:
places["geometry"].head()
# if you explore here, you notice that with to_crs, every polygon value becomes "inf"

In [ ]:
# this runs fine, but...
places.iloc[0]

In [ ]:
# this kills my kernel (Py root)
places.iloc[0]["geometry"]

In [ ]:
for geo in places["geometry"]:
    print geo[0]
    print "\n"
    print geo.bounds[0]
    break

### Compare json & geojson inputs

In [ ]:
raw_json = gpd.read_file("sample_data/shapes/nycha.json")

In [ ]:
raw_json["geometry"].head()

In [ ]:
raw_places["geometry"].head()

In [ ]:
# multipolygon vs regular polygon

raw_places["geometry"]

In [ ]:
EPSG = 2263

CRS = {
    'proj': 'latlong',
    'init': 'epsg:{:d}'.format(EPSG)
}

zones = exploded_df.to_crs(fiona.crs.from_epsg(EPSG))
zones_crs = zones.to_crs(CRS)

In [ ]:
zones_crs.head()

In [ ]:
exp_head = exploded_df.head(50)

merged_df = intersect.merge(exp_head, people)